# Effect of Informal Care on Labor Supply: A Macroeconomic Perspective

## 2018.3.12 ICES, Hosei University

### Daisuke Moriwaki

Cyberagent, Inc., AI Lab        
Keio University, School of Medicine
<daisuke.moriwaki@gmail.com>   github: daimoriwaki

In [1]:
using QuantEcon
using Plots
using Interpolations
using Distributions

# Motivation
* In 2017, it is reported taht there is 6 million care revievers in Japan by MLHW. The number is expected to increase as the baby boomers, or Dankai generations become "late elderly".
* It is unclear whether the Japanese LTCI system is sustanable in the sense of financially, which has been pointed out by several researchers.
* More importantly, the impact of care needs on economy itself (oppose to accountant's view) should be studied.


* It is easy to predict that "caregiver ratio" (the number of caregivers per one care receiver) explodes in the near future.
* Aside from the financial sustainability, the explosion of caregiver ratio cause several problems
    * insufficient care for elderly (insufficient availability of cheap nursing home, *tokuyou*)
    * Poverty due to care-leave for frail older family
    * *Economy itself* due to less labor supply
* Such a wide range of problem should be tackled by macroeconomic model rather than causal analysis.

# Summary of Results
* ...

# Literature

* Briefly survery several work in the labor and macro fields.



## Labor supply and informal care (causal studies)

* moriwaki (2015, mimeo) finds negative effect from informal care only for male.
* Nishimura
* 

## Care and Macro economy
* 




## Sustainability due to aging population in Japan
* Kitao




## Complex heterogeneous agent model concerned with social security programs
* Li (EER, 2018)

# Set up *"Care Economy"*

In [2]:
# various settings


### Economy ###

r = 0.01; # interest rate
dep_rate = 1 # dependency rate
wage_care = 0.7; # care workers' wage rate. ratio to all industry
sick_rate = 0.2; # need care old / old
care_hour = 0.3; # care demand per care receiver in hours

### Government ###
tau = 0.1; # income tax rate
rep_rate = 0.5; # replacement rate
NH_rate = 0.9; # government formal care provision rate. ratio to needed care
retirement_age = 3; # age when household can take up pension


### Preference ###

beta = 0.9; # discount rate
chi = 0.4; # share of leisure in utilty
sigma = 0.28; # intertemporal substitution

### numerical excersize ###

num_grid = 9; # number of grid usually 9
min_asset = 0; # minimum asset
max_asset = 10; # maximum asset
min_leisure = 0.001;
max_leisure = 0.999;

asset_grid = linspace(min_asset, max_asset, num_grid);
leisure_grid = linspace(min_leisure, max_leisure, num_grid);

# Demography
* 3 periods: young, middle, and old
    * young: work in the market and consume goods and leisure.Save for the future 22-44
    * middle: work both in the market and home. consume goods and leisure. Save for the future. No mortality risk. 45-64
    * old: no work in both market and home. Those with bad health receive care either in public nursing home or at home. Mortality risk is present. 65+
* Consider 3 states: now (old is less than middle and young), 10 years later (old is more than middle and young), 20 years later (old is much more than young and middle)

## Household

* The model economy is populated by many households. 
    * You may imagine either single households without gender and unknown re-production technique or many households with couples and kids where income, assets, health, and time endowment is shared. 
    * More realistic modelling is listed as future study
* wage rate is exogenous to the agent. Given wage rate, labor supply is endogeneous.
* No negative saving is allowed (consider whether we can borrow without collateral).

## Care in detail
* In this economy, every middle-aged households face the risk of becoming care giver.
* In the beginning of the period, the health status of all households is assigned. 
* Then the total *care needs* is determined. 
* The government determine how much care to provide, which is sometimes insufficient. That deliniates reality in Japan (there are 360,000 people on the waiting list).
* Unmet care needs is distributed to potential care givers (*middle*). No state dependency. Imagine that you might become a informal care giver for your uncle, spouse's siblings and so on as well as your own genetic parents and spouse. Unexpectable.
* This scenario totally igonres the positive utility from care-giving. But there are lots of tragedies associated with informal care.

## Preference
* simple and general utility function. e.g. Yue (2018, EER)

In [3]:
function calc_utility(c::Real, l::Real,chi::Real = 0.4, sigma::Real = 0.28) # chi = 0.4
    if c <= 0 || l <= 0
        return -1000
    else
        return ((c^(1-chi)*(l^chi))^(1-sigma))/(1 - sigma)
    end
end

calc_utility (generic function with 3 methods)

## consumption
* consumption is deterimined by assets, income and saving
* there is a social assistence for the poor

In [4]:
function calc_consumption(new_asset::Real, income::Real, asset::Real, r::Real=r)
    (1 + r)*asset + income - new_asset # allow negative consumption
end

calc_consumption (generic function with 2 methods)

## income
* income is determined by hourking hour in market and tax


In [5]:
function calc_income(leisure::Real, wage::Real, hour_home::Real, pension::Real, tau::Real = tau)
    (1 - tau)*(wage*(1 - leisure - hour_home) + pension)
end

calc_income (generic function with 2 methods)

## pension
* pension is taken up by old.
* the level of benefit is determined by wage history and replacement rate

In [7]:
"calc_pension(age, wage_history, rep_rate, retirement_age)

**Argments**

average_wage is meanvector of past and current wage

return 0 if younger than  retirement age"

function calc_pension(age, average_wage, rep_rate = rep_rate, retirement_age = retirement_age)
    if age >= retirement_age
        return rep_rate*mean(average_wage)
    else
        return 0
    end
end

calc_pension

## wage
* wage evolves according to $w_{i,t} = \mu + \rho w_{i, t-1} + \epsilon_{t}$. 
* need to be complicated when finer horizon is embedded.
* use tauchen method to generate transition matrix

In [8]:
# wage trainsition matrix

wage_rho = 0.9 # param for wage function
wage_sigma = 0.1# param for wage function standard deviation for epsilon
wage_mu = 0 # mean wage
wage_n_std = 9 # length of grid

wage_markovchain = tauchen(num_grid, wage_rho, wage_sigma, wage_mu, wage_n_std);
wage_trans_mat = wage_markovchain.p;
wage_grid = exp(wage_markovchain.state_values);

In [9]:
# average wage transition
# only applicable when number of period = 3 NO SCALABILITY!!!!
average_wage_trans_mat = wage_trans_mat/2 + eye(num_grid, num_grid)/2;

## Informal care
* Middle-aged household is assigned care needs randomly.
* The level of care is represented as required hour for care at home.
* Once all the household draw the raw hour at home $h^H$, they are sum up as $\sum_i h^H_i$
* The total demand for care $D^H = \sum_i I (h_i \in (0,  0.1)$
* The unmet care needs $D^{H, unmet} = D^H - NH$ is distributed out to proportionaly to $h^H$
* Numerical excercise is done without empirical studies, which should be done in the future.

In [10]:
# informal care grid and transition matrix

hour_home_rho = 0 # param for wage function
hour_home_sigma = 0.2# param for wage function standard deviation for epsilon
hour_home_mu = 0 # mean wage
hour_home_n_std = 9 # length of grid

hour_home_markovchain = tauchen(num_grid, hour_home_rho, hour_home_sigma, hour_home_mu, hour_home_n_std);
hour_home_trans_mat = hour_home_markovchain.p;
hour_home_grid = exp(hour_home_markovchain.state_values);

In [11]:
" calc_hour_home(relative_hour_home, dep_rate, sick_rate, care_hour,NH_rate)

calculate hour at home given relative home hour. relative home hour represent the level of informal care giving 
"
function calc_hour_home(relative_hour_home::Real, dep_rate::Real=dep_rate, sick_rate::Real=sick_rate, care_hour::Real=care_hour,
        NH_rate::Real = NH_rate)
    relative_hour_home*(1 - NH_rate)*dep_rate*sick_rate*care_hour
end


calc_hour_home

## health status
* health status so far only matters how many old need care.
* health is not associated with wage, labor.
* we do not take health into accont explicitly.

## labor supply
* labor supply is endogeneous
* young and middle household optimizes labor supply as well as consumption (saving) for given time endowment
* time endowment is varying across agent due to informal care
* From the first order condition $l = c(1 - \tau)w \frac{\chi}{1 - \chi}$

In [12]:
"calc_hour_mkt(age, leisure, hour_home, retirement_age)

calculate hour in the market.

**Argments**: 

        age, leisure, hour at homt

**Return**

        working hour at market"
function calc_hour_mkt(age::Real, leisure::Real, hour_home::Real, retirement_age::Int=retirement_age)
    if leisure + hour_home >= 1 || age >= retirement_age
        return 0
    else
        return 1 - leisure - hour_home
    end
end

calc_hour_mkt

## Government
* government provide insufficient formal care funded by social security premium contributed by working age, which is resemble to Japanese LTCI
* cost of formal care is determined by personal expenses of informal care worker

# Quantitative Exercise

## Solution
* The model is solved (ideally) by method of simulated moment.
* So far no calibration made
* Optimization by households is done by backward induction.

## Value function
* value function optimization $v_t = max_{c_t, l_t} (u(c_t, l_t) + \beta \mathbb{E}\left( v_{t+1}(a_{t+1}) \right))$ 

In [13]:
"value_function(utility, expected_value, beta)
calculate instant value for given utility and expected values in the next period"
function value_function(utility::Real, expected_value::Real, beta::Real = beta)
    utility + beta*expected_value
end

value_function

## Expected values
* expected values are dot product of transition matrix and state-dependent values 

In [14]:
function interpolate_vector(current_value::Real, grid, trans_mat, num_grid = num_grid)
    itp = interpolate((grid, grid), trans_mat, Gridded(Linear()))
    vec = itp[current_value, grid]
    for i in 1:num_grid
        if vec[i] < 0
            vec[i] = 0
        end
    end
    return vec
end

interpolate_vector (generic function with 2 methods)

In [15]:
"gen_prob_vec
return probability vector for arbitrary wage, hour at home, current_average_wage

**Args**
current_wage
current_hour_home
current_average_wage

**return**
vector of probability
"
function gen_prob_vec(current_wage, current_hour_home, current_average_wage, wage_grid = wage_grid, 
        hour_home_grid = hour_home_grid, wage_trans_mat = wage_trans_mat, average_wage_trans_mat = average_wage_trans_mat,
        hour_home_trans_mat= hour_home_trans_mat)
    average_wage_prob_vector = interpolate_vector(current_average_wage, wage_grid, average_wage_trans_mat)
    wage_prob_vector = interpolate_vector(current_wage, wage_grid, wage_trans_mat)
    hour_home_prob_vector = interpolate_vector(current_hour_home, hour_home_grid, hour_home_trans_mat)
    return kron(kron(wage_prob_vector, hour_home_prob_vector), average_wage_prob_vector)
end

gen_prob_vec

In [16]:
gen_prob_vec(1,1,1)

1×729 Array{Float64,2}:
 7.29351e-161  5.28204e-126  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [19]:
"get_new_value(new_asset, current_wage, current_hour_home, current_average_wage, max_value_mat): 


get vector of maximized value for next period given choice asset and states current wage, current hour at home. 
max_value_mat should be asset_grid x wage, relative_hour_home x average_wage"
function get_new_value(new_asset, current_wage, current_hour_home, current_average_wage, 
        max_value_mat, asset_grid = asset_grid, num_grid = num_grid)
    v_new = zeros(num_grid^3, 1)
    @simd for i in 1:num_grid^3
        knots = (asset_grid, )
        itp = interpolate(knots, max_value_mat[:,i], Gridded(Linear()))
        v_new[i] = itp[new_asset]
    end
    return v_new
end

get_new_value

## Grid Search

In [20]:
"generate_max_values_matrix(value_function, asset_grid, leisure_grid, wage_grid, hour_home_grid, num_grid):

maximize value with respect to leisure and consumption
"

function generate_max_values_matrix(age, value_function::Function, new_max_value_mat = zeros(num_grid, num_grid^3), asset_grid = asset_grid, leisure_grid = leisure_grid, 
        wage_grid = wage_grid, hour_home_grid = hour_home_grid, num_grid = num_grid)
    # initializing matrices
    value_matrix = zeros(num_grid^2, num_grid^4);
    consumption_mat = zeros(num_grid^2, num_grid^4);
    leisure_mat = zeros(num_grid^2, num_grid^4);
    income_mat = zeros(num_grid^2, num_grid^4);
    new_asset_mat = ones(num_grid^2, num_grid^4);
    wage_mat = zeros(num_grid^2, num_grid^4);
    asset_mat = zeros(num_grid^2, num_grid^4);
    hour_home_mat = zeros(num_grid^2, num_grid^4); # relative hour home (not real one)
    average_wage_mat = zeros(num_grid^2, num_grid^4);
    @simd for new_asset_ind = 1:num_grid
        for leisure_ind = 1:num_grid
            for asset_ind = 1:num_grid
                for wage_ind = 1:num_grid
                    for hour_home_ind = 1:num_grid
                        for average_wage_ind = 1:num_grid
                            new_asset = asset_grid[new_asset_ind]      
                            leisure = leisure_grid[leisure_ind]
                            wage = wage_grid[wage_ind]
                            asset = asset_grid[asset_ind]
                            raw_hour_home = hour_home_grid[hour_home_ind]
                            hour_home = calc_hour_home(raw_hour_home)
                            average_wage = wage_grid[average_wage_ind]
                            pension = calc_pension(age, average_wage)
                            income = calc_income(leisure, wage, hour_home, pension)
                            consumption = calc_consumption(new_asset, income, asset)
                            utility = calc_utility(consumption, leisure)
                            
                            # calc expected_value
                            v_new_vec = get_new_value(new_asset, wage, hour_home, average_wage, new_max_value_mat)
                            p = gen_prob_vec(wage, hour_home, average_wage)
                            expected_value = dot(v_new_vec, p)
                            value_matrix[new_asset_ind + (leisure_ind - 1)*num_grid, 
                                asset_ind + (wage_ind - 1)*num_grid + (hour_home_ind - 1)*num_grid^2 + 
                                (average_wage_ind - 1)*num_grid^3] = value_function(utility, expected_value)
                        end
                    end
                end
            end
        end
    end
    # get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
    #max_value = maximum(value_matrix, 1)
    max_value, max_ind = findmax(value_matrix, 1)
    # reshape value matrix to asset_grid x (w x h_H)
    # this should be fix throughout the imulation
    max_values_reshaped = reshape(max_value, num_grid, num_grid^3);
    return max_values_reshaped, max_ind
end



generate_max_values_matrix

In [ ]:
age = 3
max_value_matrix, max_ind = generate_max_values_matrix(age, value_function);

In [ ]:
plot(max_value_matrix[1,:])

In [ ]:
age = 2;
max_value_matrix, max_ind = generate_max_values_matrix(age, value_function, max_value_matrix);

In [ ]:
plot(max_value_matrix[1,:])

In [ ]:
age = 1;
max_value_matrix, max_ind = generate_max_values_matrix(age, value_function, max_value_matrix);

In [ ]:
plot(max_value_matrix[1,:])